# Design of exon capture probes from _Trifolium repens_ transcriptome
### **Author:** James S. Santangelo
### **Date:** August 13<sup>th</sup>, 2018

This pipeline is is meant for designing exon capture probes from transcript sequences using the reference genome of a closely related species. 

## Step 1: Downloading transcriptome and reference genome

* The white clover transcriptome was downloaded from [BMC Genomics](https://bmcgenomics.biomedcentral.com/articles/10.1186/1471-2164-14-100#Sec19), which also includes the description of the transcriptome generated by Nagy *et al.* (2014).
* The *Trifolium pratense* reference genome and annotation files (both *.gtf* and *.gff3* files) were downloaded from [Ensembl plants](https://plants.ensembl.org/info/website/ftp/index.html) (release 40, July 2018) on July 31,<sup>th</sup>, 2018. The genome was published by [De Vega *et al.* (2015)](https://www.nature.com/articles/srep17394) and full assembly details are available on the [European Nucleotide Archive](https://www.ebi.ac.uk/ena/data/view/GCA_900079335.1).

## Step 2: Retrieving exons from the _Trifolium pratense_ reference genome

* Prior to any filtering, the _T. pratense_ genome contained **184,849 unique exons** from **39,948 unique genes**
* Using the *.gff3* annotation file and the `gffutils` python module, I extracted the sequence of each of these exons and wrote them to a new FASTA file. The first 5 sequences in the exon FASTA are shown below,which represent the first 5 exons from the gene with ID 4341

In [8]:
%%bash

less ../../example_outputs/pratenseExons_unfiltered_First5.txt

>Tp57577_TGAC_v2_mRNA4484.exon1.gene4341
CAGCGAATGAACGAGGTTCTCCCACACAGCGAACCGCATTCTCCCTCAGCGAACCCTAAGTCGTTCTTCCTCTCGGTGAAATGCAAATTATGTCCCTCTCATGCTGCGACGAAATCGATTACGGCGACCACGTTCTGCTACCACGGTGAAATTCGTCCTcccccccTTTCAAG
>Tp57577_TGAC_v2_mRNA4484.exon2.gene4341
AAGTAGTTATGGATCGTAGTTGGATGAATGCTAACCAGTTCAGTGAAGAGTATAAGAAAGAG
>Tp57577_TGAC_v2_mRNA4484.exon3.gene4341
GCTTGTAATGAATACCACCACATCGTGTCTCGTGGGGGATATGAGTTGATTGAAGATAAGATGTTGCAAGATGAAATTAAACAAAGACAAAAGTCAGCTGGAGATTCCCTACCTACACCTCCATCTTCACCAAAACGCTATGAGAAGTGGATAAGGGGACGAAAAAGCATATGTATCAGAAGGATAAATG
>Tp57577_TGAC_v2_mRNA4484.exon4.gene4341
aagttgaggctgaaactagaattgacaatgttgagactgaaaTTGATCCTAAAAATGTTGAAACTAGAGGTGACAATGTTGAGACTGAAATTGATTCTGAAAAAGATGGAAATGTTGGTGTTGATGTAAGATATCGGAAGAATTTAGTATACACAAGGAAAAAGAATATCATTCCTGAATnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnTAGTGATTCAATTTCCGAATTTTCTCCtgctccagttgcaaaaatgaatact
>Tp57577_TGAC_v2_mRNA4484.exon5.gene4341
catcttggaagacttgaggataaagagtgatgaaccaatgaaatttattgtgataataaatctgctattagcat


* The unfiltered _T. pratense_ exons then had to be filtered. I imposed the following two filtering criteria:

    1) The exons could not contain any ambiguous nucleotides (i.e. 'n' or 'N').
    
    2) The exons had to be greater than 200 bp in length since evidence suggests that the capture efficiency of probes declines for sequences shorter than 200 bp (see [Bragg *et al.* (2016)](https://www.ncbi.nlm.nih.gov/pubmed/26215687))
<br>
<br>
* Following filtering, there were **72,355 unique exons** remaining from **35,109 unique genes**.

## Step 3: Reciprocal best blast

* In this step, I perform a reciprocal best blast between *T. repens* transcripts and the filtered *T. pratense* reference exons.
<br>
<br>
First, I Blasted the transcriptome against the exon sequences using the following command:
<br>
<br>
`blastn -query Transcriptome.fasta -db Exons.db -out <OUTFILE> -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend qlen sstart send slen evalue bitscore qcovs qcovhsp' -num_threads 2 -evalue 1e-10 -max_target_seqs 1 -max_hsps 1`
<br>
<br>
`-evalue 1e-10` Keep only hits where Evalue is less than 1e<sup>-10</sup> <br>
`-max_target_seq` Ensure that each transcript hits at most 1 exon target (i.e. the best one) <br>
`-max_hsps` Ensure that only the top hit for each target sequence is included in the final output. 
<br>
<br>
* This approach resulted in **43,714 hits**. The first 10 hits are shown in the dataframe below as an example.

In [6]:
import pandas as pd

colnames = ["qseqid", "sseqid", "pident", "length",
            "mismatch", "gapopen", "qstart", "qend",
            "qlen", "sstart", "send", "slen", "evalue",
            "bitscore", "qcovs", "qcovhsp"]

transcriptome_to_exons = pd.read_table("../../image/recipBlast_Trans2Exons_First10.txt",
                                       sep="\t",
                                       names=colnames)
transcriptome_to_exons

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,qlen,sstart,send,slen,evalue,bitscore,qcovs,qcovhsp
0,wcd-8466,Tp57577_TGAC_v2_mRNA3036.exon6.gene2951,97.319,373,9,1,1,373,999,372,1,475,1.260000e-180,632,37,37
1,wcd-7834,Tp57577_TGAC_v2_mRNA12970.exon3.gene12562,92.000,450,34,2,418,867,999,1,448,472,4.550000e-180,630,45,45
2,wcd-7404,Tp57577_TGAC_v2_mRNA22327.exon1.gene21589,95.349,129,6,0,871,999,999,550,422,550,3.270000e-52,206,13,13
3,wcd-42747,Tp57577_TGAC_v2_mRNA26352.exon1.gene25485,94.316,563,21,4,403,965,999,639,88,639,0.000000e+00,852,56,56
4,wcd-41619,Tp57577_TGAC_v2_mRNA10895.exon4.gene10542,96.035,227,9,0,479,705,999,227,1,227,1.090000e-101,370,23,23
5,wcd-3612,Tp57577_TGAC_v2_mRNA13531.exon1.gene13095,96.070,458,13,3,31,483,999,135,592,592,0.000000e+00,741,45,45
6,wcd-1589,Tp57577_TGAC_v2_mRNA9035.exon4.gene8741,97.568,329,5,2,200,528,999,326,1,326,6.050000e-159,560,33,33
7,wcd-13378,Tp57577_TGAC_v2_mRNA39072.exon1.gene37796,90.845,710,38,12,23,729,999,313,998,998,0.000000e+00,929,71,71
8,wcc-26375,Tp57577_TGAC_v2_mRNA27039.exon1.gene26146,95.929,1007,30,9,1,999,999,1505,502,1625,0.000000e+00,1624,100,100
9,wcc-14061,Tp57577_TGAC_v2_mRNA17836.exon6.gene17237,96.082,485,19,0,180,664,999,1,485,485,0.000000e+00,800,49,49


* I then used the same approach to BLAST the filtered reference exons against the transcript sequences, which resulted in **39,973 hits**.
* I identified reciprocal hits by matching the query and subject sequences from the 2 independent BLAST results. A hit was identified as reciprocal if the query sequence/subject sequence pairs were the same in both datasets (i.e. a given exon is the best match to a given transcript in the first BLAST and that same transcript is the best match to the same exon in the second BLAST).Using this approach, I identified **22,121 reciprocal hits**.
* I then filtered these reciprocal hits using the following criteria:

    1) For the same reason as above, the length of the alignment had to be > 200 bp.
    
    2) No two exons could come from the same gene
        * Where a gene had more than one exon, I selected the one with the highest bitscore. If more than one exon had the same bitscore, I selected the one with the greatest % identity to the transcript sequence.  
    
    3) The % identity of the transcript-exon alignment had to be greater than 80%.
    
    4) More than 50% of the exon had to be covered by transcript sequence. 
    
* Following filtering, **10,176 reciprocal hits** remained. These are the hits that were used to extract sequences from the transcriptome for which capture probes will be designed.

## Step 3: Sequences for probe design

* The sequences for probe design were extracted from the _T. repens_ transcriptome using the query sequence start and end coordinates from the reciprocal hits data frame, which is identical to the table shown above but has been filtered. 
* Importantly, I took care to ensure that strandedness was maintained.
    * For sequences on the + strand (i.e. start coordinate < end coordinate), the sequences could be extracted directly from the transcriptome using these coordinates. 
    * For sequences on the – strand (i.e. end coordinate < start coordinate), the start and end coordinates had to be swapped and the resulting sequence was reverse complemented. 
* While extracting the sequence, I additionally extracted the transcript annotation and place this in the FASTA header for easy querying (see below).
* Finally, I only kept sequences if their % GC content was between 30% and 70% since sequences with low or high GC content have reduced capture efficiency (see [Bi _et al._ (2016)](https://bmcgenomics.biomedcentral.com/articles/10.1186/1471-2164-13-403)). The final, filtered FASTA file has **10,121 sequences**, covering approximately **5.29 Mbp** of sequence. The first 5 sequences from the FASTA file are shown below. 

In [7]:
%%bash

less ../../image/exonTargets_First5.txt

>Transcript_id:wcd-8855;Strand:-;PratenseExon_id:Tp57577_TGAC_v2_mRNA4492.exon1.gene4349;Length:1033;Transcript_annot:no annotation
GAAGCTAAATACATGATACTAAAAAATCTGATGGAAGAAAAACAGTTGGATTTTAATCAGCCACTTTTATCAGTAAGGCGATTCCCATCAACAGCACCATACTGAAAGACGACAAGTAAAAGGAAAACCGATAAGCCAATATCGAAGAGAACTCGTCTTCCTGCTTACAAATCAGAAGTTAAAWWCAGGTCCGGTGACTAACCCTGGAACTGTTCTTTCGTATGGGAGAAGACTCCCGGAAGACCTATAGATGAAAGCAAAGYKTCAACAAYGGCTATYGAAGGACCTCTYGTCGCTCCTAAGCTTCCACCKGGAAGGGTTTTAAAAGTTGAACAGCAAGATTTTGATAAAACTCCTAAGGGCTCATTGGTTACTCGGTCCAGAACAGAAAGCACTGTTTCCAACTCTATGAGTATTGCATCTTTGGATAGTAAAGAAGCAAATCATGACAGCCGAAAAGAAGAGATTCTGGAAAAGGAAAGTTCTGGTTCAGATAATGGGGATGAGACCTATCTAGATGCTCGTGATACACTTTCTAGAACCGAATCATTCTTCATGAACTGCAGTGTGAGTGGTATGAGTGGATGTGATGATCGAGAGGTTCAACCATCCGAAACTTTTTCGGCAGATCAACAGGCTCGTGATTTCATGATCGGTAGGTTTTTGCCTGCAGCAAAGGCTATGGCTTCTGAAACGCCTCATGTCCAATACGCCTCTAGGAAGCCAATTGTTAGACAGGAACAACCTAGACTAGTAAGGAAAGTAGAAAGTGGAGCCAAGTCCCGTCCCTTGGACCCGAAATGGCAAAAGGTTTTGCCACATTATGCACAAGATACGAGCCTCAATGAAAGTGAAGACGAAAGCGACG

## Transcript annotations

* Because we are interested in including sequences underlying proteins that may be influenced by urbanization, I queried the transcript sequence annotations for certain abiotic/biotic factor that may be influenced by cities. Specifically, I looked for:
    * salt (6 hits)
    * drought (2 hits)
    * nodul* (35 hits)
    * cyanog* (2 hits): A cyanogenic beta-glucosidase and a non-cyanogenic beta-glucosidase
    * cold (1 hit)
    * stress (22 hits)
    * dehydrin (involved in drought response, 1 hit)
    
The above queries resulted in a combined **68 hits**. 

Also, note that **1,642 sequences** have no annotation. 

## Additional considerations

1) We still have ~0.7 Mbp worth of sequence for which we can design probes since the kit we are buying comes in a 6 Mbp size. How should we target additional sequences?

* First, we can add sequences for regions we are interested in using the white clover EST/transcript database. For example, only linamarase was a hit to the transcriptome but I suggest we design probes for all _CYP79D15_ and _Li_ sequences that are currently published, including flanking regions. However, this will only be a fraction of the 0.7 Mbp. 
* We could also just design 200, 400, 600 and 800 bp probes for random stretches of transcript sequences, targeting only those that were not homologous to red clover exons. While not homologous to red clover exons, these transcripts still represent sequences being expressed by white clover and may be sequences unique to white clover. While some of these probes risk overlapping exon boundaries and not binding, I don't think this will be much of an issue.

2) Should we BLAST the final set of exons against itself to check for repetitive segments? My sense is that we don't have to since the probe design company screens for repeats as part of their pipeline. 